In [1]:
import yaml
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from linkedineasyapply import LinkedinEasyApply
from validate_email import validate_email

def init_browser():
    browser_options = Options()
    optionss = ['--disable-blink-features', '--no-sandbox', '--start-maximized', '--disable-extensions',
               '--ignore-certificate-errors', '--disable-blink-features=AutomationControlled', '--remote-debugging-port=9222']

    for option in optionss:
        browser_options.add_argument(option)

    driver = webdriver.Chrome(options=browser_options)

    driver.set_window_position(0, 0)
    driver.maximize_window()

    return driver


def validate_yaml():
    with open("config.yaml", 'r') as stream:
        try:
            parameters = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            raise exc

    mandatory_params = ['email', 'password', 'disableAntiLock', 'remote', 'jobTypes', 'date',
                        'positions','employee count', 'outputFileDirectory', 'checkboxes', 'universityGpa',
                        'languages', 'experience', 'personalInfo', 'eeo', 'uploads']

    for mandatory_param in mandatory_params:
        if mandatory_param not in parameters:
            raise Exception(mandatory_param + ' is not inside the yml file!')

    assert validate_email(parameters['email'])
    assert len(str(parameters['password'])) > 0

    assert isinstance(parameters['disableAntiLock'], bool)

    assert isinstance(parameters['remote'], bool)

    assert len(parameters['jobTypes']) > 0
    job_types = parameters.get('jobTypes', [])
    at_least_one_job_type = False
    for key in job_types.keys():
        if job_types[key]:
            at_least_one_job_type = True
    assert at_least_one_job_type

    assert len(parameters['date']) > 0
    date = parameters.get('date', [])
    at_least_one_date = False
    for key in date.keys():
        if date[key]:
            at_least_one_date = True
    assert at_least_one_date

    assert len(parameters['positions']) > 0

    assert len(parameters['uploads']) >= 1 and 'resume' in parameters['uploads']

    assert len(parameters['employee count']) > 0

    assert len(parameters['checkboxes']) > 0

    checkboxes = parameters.get('checkboxes', [])
    assert isinstance(checkboxes['driversLicence'], bool)
    assert isinstance(checkboxes['requireVisa'], bool)
    assert isinstance(checkboxes['legallyAuthorized'], bool)
    assert isinstance(checkboxes['urgentFill'], bool)
    assert isinstance(checkboxes['commute'], bool)
    assert isinstance(checkboxes['backgroundCheck'], bool)
    assert 'degreeCompleted' in checkboxes

    assert isinstance(parameters['universityGpa'], (int, float))

    languages = parameters.get('languages', [])
    language_types = {'none', 'conversational', 'professional', 'native or bilingual'}
    for language in languages:
        assert languages[language].lower() in language_types

    experience = parameters.get('experience', [])

    for tech in experience:
        assert isinstance(experience[tech], int)
    assert 'default' in experience

    assert len(parameters['personalInfo'])
    personal_info = parameters.get('personalInfo', [])
    for info in personal_info:
        assert personal_info[info] != ''

    assert len(parameters['eeo'])
    eeo = parameters.get('eeo', [])
    for survey_question in eeo:
        assert eeo[survey_question] != ''

    return parameters


In [2]:
parameters = validate_yaml()
browser = init_browser()

bot = LinkedinEasyApply(parameters, browser)

In [3]:
import time, random, csv, pyautogui, pdb, traceback, sys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from datetime import date
from itertools import product

In [4]:
bot.login()

In [5]:
self = bot
page_sleep = 0
minimum_time = 60*15
minimum_page_time = time.time() + minimum_time
position = self.positions[0]

job_page_number = -1

print("Starting the search for " + position + ".")


page_sleep += 1
job_page_number += 1
print("Going to job page " + str(job_page_number))
self.next_job_page(position,job_page_number)
time.sleep(random.uniform(1.5, 3.5))
print("Starting the application process for this page...")


Starting the search for Data Scientist.
Going to job page 0


Starting the application process for this page...


In [6]:
no_jobs_text = ""
try:
    job_results = self.browser.find_element(By.CLASS_NAME, "jobs-search-results-list")
    self.scroll_slow(job_results)
    self.scroll_slow(job_results, step=1000, reverse=True)

    job_list = self.browser.find_elements(By.CLASS_NAME, 'scaffold-layout__list-container')[0].find_elements(By.CLASS_NAME, 'jobs-search-results__list-item')
except:
    raise Exception("No more jobs on this page")


In [10]:
job_tile = job_list[20]
job_el = job_tile.find_element(By.CLASS_NAME, 'job-card-list__title')
job_el.click()

In [7]:
bot.start_applying()

Starting the search for Data Scientist.
Going to job page 0
Starting the application process for this page...
Job contains blacklisted keyword or experience does not match!
greater employee match found. Saving!
Job contains blacklisted keyword or experience does not match!
Job contains blacklisted keyword or experience does not match!
greater employee match found. Saving!
greater employee match found. Saving!
Job contains blacklisted keyword or experience does not match!
Job contains blacklisted keyword or experience does not match!
Job contains blacklisted keyword or experience does not match!
Job contains blacklisted keyword or experience does not match!
Job contains blacklisted keyword or experience does not match!
Job contains blacklisted keyword or experience does not match!
Job contains blacklisted keyword or experience does not match!
Job contains blacklisted keyword or experience does not match!
greater employee match found. Saving!
Job contains blacklisted keyword or experienc

In [ ]:
parameters = validate_yaml()
browser = init_browser()

bot = LinkedinEasyApply(parameters, browser)
bot.login()
bot.security_check()
bot.start_applying()


In [5]:
bot.start_applying()

Starting the search for Data Scientist.
Going to job page 0
Starting the application process for this page...
Job contains blacklisted keyword or company or poster name!
Done applying to the job!
Could not write the job to the file! No special characters in the job title/company is allowed!


Traceback (most recent call last):
  File "d:\EasyApplyBot-master\linkedineasyapply.py", line 228, in apply_jobs
    self.write_to_file(company, job_title, link, job_location,poster,job_employee_count)
TypeError: LinkedinEasyApply.write_to_file() missing 1 required positional argument: 'search_location'


Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or company or poster name!
Job contains blacklisted keyword or comp